In [1]:
# ══════════════════════════════════════════════════════════════════
# NOTEBOOK 01: INGESTA DE DATOS
# Disney Data Pipeline - Fase 1
# ══════════════════════════════════════════════════════════════════

import os
import json
import pickle
import time
from pathlib import Path
from datetime import datetime

import pandas as pd
import numpy as np
import requests

from dotenv import load_dotenv
import boto3
from botocore.exceptions import ClientError, NoCredentialsError

import warnings
warnings.filterwarnings('ignore')

print("✅ Imports completados")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Imports completados
📅 2025-10-17 16:38:07


In [2]:
# ══════════════════════════════════════════════════════════════════
# CELDA 2: CARGAR VARIABLES DE AMBIENTE
# ══════════════════════════════════════════════════════════════════

load_dotenv(override=True)

required = {
    'AWS_ACCESS_KEY_ID': 'AWS Access Key',
    'AWS_SECRET_ACCESS_KEY': 'AWS Secret Key',
    'AWS_DEFAULT_REGION': 'AWS Region',
    'S3_BUCKET_NAME': 'S3 Bucket'
}

print("🔍 Verificando configuración:\n")
missing = []

for var, desc in required.items():
    value = os.getenv(var)
    if value:
        masked = f"{value[:4]}...{value[-4:]}" if 'KEY' in var else value
        print(f"✅ {desc:20} : {masked}")
    else:
        print(f"❌ {desc:20} : NO CONFIGURADA")
        missing.append(var)

if missing:
    raise EnvironmentError(f"❌ Faltan variables: {missing}")

print("\n✅ Variables configuradas correctamente")

🔍 Verificando configuración:

✅ AWS Access Key       : AKIA...63PU
✅ AWS Secret Key       : mNaj...3rkr
✅ AWS Region           : us-west-1
✅ S3 Bucket            : xideralaws-curso-fernanda

✅ Variables configuradas correctamente


In [3]:
# ══════════════════════════════════════════════════════════════════
# CELDA 3: CREAR SESIÓN AWS SEGURA
# ══════════════════════════════════════════════════════════════════

def get_aws_session():
    """Crea sesión AWS segura con validación"""
    try:
        session = boto3.Session(
            aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
            aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
            region_name=os.getenv('AWS_DEFAULT_REGION')
        )
        
        # Verificar credenciales con STS
        sts = session.client('sts')
        identity = sts.get_caller_identity()
        
        print("✅ Sesión AWS creada")
        print(f"   Account ID: {identity['Account']}")
        print(f"   User ARN: {identity['Arn']}")
        print(f"   Region: {session.region_name}")
        
        return session
    
    except NoCredentialsError:
        raise Exception("❌ Credenciales AWS no encontradas")
    except ClientError as e:
        raise Exception(f"❌ Error de autenticación: {e}")

# Crear sesión y clientes
aws_session = get_aws_session()
s3_client = aws_session.client('s3')

# Configuración S3
S3_BUCKET = os.getenv('S3_BUCKET_NAME')
S3_RAW_PREFIX = 'disney-project/raw'
S3_CLEANED_PREFIX = 'disney-project/cleaned'

print(f"\n✅ S3 configurado")
print(f"   Bucket: {S3_BUCKET}")
print(f"   Prefix Raw: {S3_RAW_PREFIX}")

✅ Sesión AWS creada
   Account ID: 020635523025
   User ARN: arn:aws:iam::020635523025:user/consola
   Region: us-west-1

✅ S3 configurado
   Bucket: xideralaws-curso-fernanda
   Prefix Raw: disney-project/raw


In [4]:
# ══════════════════════════════════════════════════════════════════
# CELDA 4: FUNCIÓN PARA SUBIR ARCHIVOS A S3
# ══════════════════════════════════════════════════════════════════

def upload_to_s3(local_file, s3_key, content_type='text/csv'):
    """
    Sube un archivo local a S3 con encriptación AES256
    
    Args:
        local_file (str): Ruta del archivo local
        s3_key (str): Key en S3 (path completo)
        content_type (str): Tipo de contenido
    
    Returns:
        str: Mensaje de resultado
    """
    try:
        if not os.path.exists(local_file):
            return f"❌ Archivo no encontrado: {local_file}"
        
        s3_client.upload_file(
            Filename=local_file,
            Bucket=S3_BUCKET,
            Key=s3_key,
            ExtraArgs={
                'ServerSideEncryption': 'AES256',
                'ContentType': content_type
            }
        )
        
        file_size = os.path.getsize(local_file) / 1024  # KB
        return f"✅ Subido: s3://{S3_BUCKET}/{s3_key} ({file_size:.1f} KB)"
        
    except ClientError as e:
        return f"❌ Error AWS: {e}"
    except Exception as e:
        return f"❌ Error: {e}"

print("✅ Función upload_to_s3 definida")

✅ Función upload_to_s3 definida


In [5]:
# ══════════════════════════════════════════════════════════════════
# CELDA 5: CREAR ESTRUCTURA DE DIRECTORIOS
# ══════════════════════════════════════════════════════════════════

directories = [
    'data/raw/kaggle',
    'data/raw/api',
    'data/cleaned',
    'data/final'
]

print("📁 Creando estructura de directorios...")
for directory in directories:
    Path(directory).mkdir(parents=True, exist_ok=True)
    print(f"   ✅ {directory}")

print("\n✅ Directorios creados")

📁 Creando estructura de directorios...
   ✅ data/raw/kaggle
   ✅ data/raw/api
   ✅ data/cleaned
   ✅ data/final

✅ Directorios creados


In [7]:
# ══════════════════════════════════════════════════════════════════
# CELDA 6: CARGAR DATASET KAGGLE - PELÍCULAS DISNEY 
# ══════════════════════════════════════════════════════════════════

print("📥 CARGANDO DATASET DE KAGGLE\n")
print("=" * 80)


kaggle_file = 'Case Study Data 2024.csv' 

# Si el archivo está en otra carpeta, usa la ruta completa:
# kaggle_file = '/ruta/completa/Case Study Data 2024.csv'

# Verificar que existe
if not os.path.exists(kaggle_file):
    print(f"❌ ERROR: Archivo no encontrado: {kaggle_file}")
    print("\n💡 Ruta actual de trabajo:")
    print(f"   {os.getcwd()}")
    print("\n📁 Archivos en directorio actual:")
    for file in os.listdir('.'):
        if file.endswith('.csv'):
            print(f"   - {file}")
    df_movies = None
else:
    try:
        # Cargar CSV con encoding apropiado
        df_movies = pd.read_csv(kaggle_file, encoding='latin-1')
        
        print(f"✅ Dataset cargado exitosamente")
        print(f"   Archivo: {kaggle_file}")
        print(f"   Registros: {len(df_movies):,}")
        print(f"   Columnas: {len(df_movies.columns)}")
        
        # Mostrar columnas
        print(f"\n📋 Columnas disponibles:")
        for i, col in enumerate(df_movies.columns, 1):
            print(f"   {i:2d}. {col}")
        
        # Mostrar info
        print(f"\n📊 Información del dataset:")
        df_movies.info()
        
        # Mostrar muestra
        print(f"\n🎬 Primeras 3 películas:")
        print(df_movies.head(3))
        
        # Guardar localmente con nombre estándar
        local_raw_path = 'data/raw/kaggle/disney_movies.csv'
        df_movies.to_csv(local_raw_path, index=False, encoding='utf-8')
        print(f"\n💾 Guardado localmente: {local_raw_path}")
        
    except Exception as e:
        print(f"❌ Error al cargar CSV: {e}")
        import traceback
        traceback.print_exc()
        df_movies = None

📥 CARGANDO DATASET DE KAGGLE

✅ Dataset cargado exitosamente
   Archivo: Case Study Data 2024.csv
   Registros: 119
   Columnas: 9

📋 Columnas disponibles:
    1. film_title
    2. brand
    3. box_office_revenue
    4. opening_revenue
    5. release_date
    6. opening_revenue_over_total_revenue
    7. imdb_score
    8. rt_critics_score
    9. rt_audience_score

📊 Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   film_title                          119 non-null    object 
 1   brand                               119 non-null    object 
 2   box_office_revenue                  119 non-null    object 
 3   opening_revenue                     119 non-null    object 
 4   release_date                        119 non-null    object 
 5   opening_revenue_over_total_revenue  119 non

In [8]:
# ══════════════════════════════════════════════════════════════════
# CELDA 7: SUBIR PELÍCULAS A S3
# ══════════════════════════════════════════════════════════════════

if df_movies is not None:
    print("☁️  SUBIENDO PELÍCULAS A S3\n")
    print("=" * 80)
    
    # Subir archivo
    s3_key = f"{S3_RAW_PREFIX}/kaggle/disney_movies.csv"
    result = upload_to_s3(local_raw_path, s3_key, 'text/csv')
    print(result)
    
    # Verificar en S3
    try:
        response = s3_client.head_object(Bucket=S3_BUCKET, Key=s3_key)
        print(f"\n🔍 Verificación S3:")
        print(f"   Tamaño: {response['ContentLength'] / 1024:.1f} KB")
        print(f"   Encriptación: {response.get('ServerSideEncryption', 'N/A')}")
        print(f"   Última modificación: {response['LastModified']}")
        print(f"   ✅ Archivo verificado en S3")
    except Exception as e:
        print(f"⚠️  No se pudo verificar: {e}")
else:
    print("❌ No hay datos de películas para subir")

☁️  SUBIENDO PELÍCULAS A S3

✅ Subido: s3://xideralaws-curso-fernanda/disney-project/raw/kaggle/disney_movies.csv (10.2 KB)

🔍 Verificación S3:
   Tamaño: 10.2 KB
   Encriptación: AES256
   Última modificación: 2025-10-17 16:54:03+00:00
   ✅ Archivo verificado en S3


In [9]:
# ══════════════════════════════════════════════════════════════════
# CELDA 8: OBTENER PERSONAJES DESDE DISNEY API
# ══════════════════════════════════════════════════════════════════

print("🌐 OBTENIENDO PERSONAJES DESDE DISNEY API\n")
print("=" * 80)

# Configuración API
BASE_URL = "https://api.disneyapi.dev/character"
MAX_PAGES = 30  # Limitar para tiempo (API tiene ~150 páginas)

all_characters = []
successful_pages = 0
failed_pages = 0

print(f"📡 Obteniendo hasta {MAX_PAGES} páginas de personajes...")
print("Progreso: ", end="", flush=True)

for page in range(1, MAX_PAGES + 1):
    try:
        # Request a la API
        response = requests.get(
            BASE_URL, 
            params={'page': page}, 
            timeout=10
        )
        
        if response.status_code == 200:
            data = response.json()
            characters = data.get('data', [])
            all_characters.extend(characters)
            successful_pages += 1
            print("✓", end="", flush=True)
        else:
            failed_pages += 1
            print("✗", end="", flush=True)
        
        # Pausa para no sobrecargar API
        time.sleep(0.1)
        
        # Mostrar progreso cada 10 páginas
        if page % 10 == 0:
            print(f" [{page}/{MAX_PAGES}]", end="", flush=True)
        
    except Exception as e:
        failed_pages += 1
        print("✗", end="", flush=True)

print(f"\n\n✅ Descarga completada:")
print(f"   Páginas exitosas: {successful_pages}")
print(f"   Páginas fallidas: {failed_pages}")
print(f"   Total personajes: {len(all_characters):,}")

# Crear DataFrame
df_characters = pd.DataFrame(all_characters)

print(f"\n📊 DataFrame creado:")
print(f"   Registros: {len(df_characters):,}")
print(f"   Columnas: {len(df_characters.columns)}")

# Mostrar columnas
print(f"\n📋 Columnas disponibles:")
for i, col in enumerate(df_characters.columns, 1):
    print(f"   {i:2d}. {col}")

# Mostrar muestra
print(f"\n👤 Muestra de personajes:")
if 'name' in df_characters.columns:
    cols_to_show = ['name', 'films', 'tvShows'] if 'tvShows' in df_characters.columns else ['name', 'films']
    print(df_characters[cols_to_show].head(5))
else:
    print(df_characters.head(5))

🌐 OBTENIENDO PERSONAJES DESDE DISNEY API

📡 Obteniendo hasta 30 páginas de personajes...
Progreso: ✓✓✓✓✓✓✓✓✓✓ [10/30]✓✓✓✓✓✓✓✓✓✓ [20/30]✓✓✓✓✓✓✓✓✓✓ [30/30]

✅ Descarga completada:
   Páginas exitosas: 30
   Páginas fallidas: 0
   Total personajes: 1,500

📊 DataFrame creado:
   Registros: 1,500
   Columnas: 12

📋 Columnas disponibles:
    1. _id
    2. films
    3. shortFilms
    4. tvShows
    5. videoGames
    6. parkAttractions
    7. allies
    8. enemies
    9. name
   10. imageUrl
   11. url
   12. alignment

👤 Muestra de personajes:
                            name  \
0                       Achilles   
1                Abigail the Cow   
2                       Abdullah   
3  Admiral Boom and Mr. Binnacle   
4                       .GIFfany   

                                              films                 tvShows  
0                                 [Hercules (film)]  [Hercules (TV series)]  
1  [The Fox and the Hound, The Fox and the Hound 2]                      []  
2     

In [10]:
# ══════════════════════════════════════════════════════════════════
# CELDA 9: GUARDAR PERSONAJES LOCALMENTE Y SUBIR A S3
# ══════════════════════════════════════════════════════════════════

print("💾 GUARDANDO PERSONAJES\n")
print("=" * 80)

# 1. Guardar como JSON
local_json_path = 'data/raw/api/disney_characters.json'

characters_data = {
    'metadata': {
        'total_characters': len(df_characters),
        'source': 'Disney API',
        'url': BASE_URL,
        'pages_retrieved': successful_pages,
        'pages_failed': failed_pages,
        'timestamp': datetime.now().isoformat()
    },
    'data': all_characters
}

with open(local_json_path, 'w', encoding='utf-8') as f:
    json.dump(characters_data, f, indent=2, ensure_ascii=False)

print(f"✅ JSON guardado: {local_json_path}")

# Subir JSON a S3
s3_key_json = f"{S3_RAW_PREFIX}/api/disney_characters.json"
result = upload_to_s3(local_json_path, s3_key_json, 'application/json')
print(result)

# 2. Guardar como CSV
print(f"\n💾 Guardando CSV...")
local_csv_path = 'data/raw/api/disney_characters.csv'
df_characters.to_csv(local_csv_path, index=False, encoding='utf-8')
print(f"✅ CSV guardado: {local_csv_path}")

# Subir CSV a S3
s3_key_csv = f"{S3_RAW_PREFIX}/api/disney_characters.csv"
result_csv = upload_to_s3(local_csv_path, s3_key_csv, 'text/csv')
print(result_csv)

print(f"\n✅ Personajes guardados en ambos formatos (JSON y CSV)")

💾 GUARDANDO PERSONAJES

✅ JSON guardado: data/raw/api/disney_characters.json
✅ Subido: s3://xideralaws-curso-fernanda/disney-project/raw/api/disney_characters.json (710.7 KB)

💾 Guardando CSV...
✅ CSV guardado: data/raw/api/disney_characters.csv
✅ Subido: s3://xideralaws-curso-fernanda/disney-project/raw/api/disney_characters.csv (343.3 KB)

✅ Personajes guardados en ambos formatos (JSON y CSV)


In [11]:
# ══════════════════════════════════════════════════════════════════
# CELDA 10: EXPLORACIÓN INICIAL DE DATOS
# ══════════════════════════════════════════════════════════════════

print("🔍 EXPLORACIÓN INICIAL DE DATOS\n")
print("=" * 80)

# ═══════════════════════════════════════
# PELÍCULAS
# ═══════════════════════════════════════
if df_movies is not None:
    print("\n🎬 PELÍCULAS:")
    print("-" * 80)
    print(f"Total registros: {len(df_movies):,}")
    
    print(f"\n📊 Información general:")
    df_movies.info()
    
    print(f"\n📈 Estadísticas numéricas:")
    print(df_movies.describe())
    
    print(f"\n❌ Valores nulos por columna:")
    null_counts = df_movies.isnull().sum()
    null_counts = null_counts[null_counts > 0]
    if len(null_counts) > 0:
        print(null_counts)
    else:
        print("   ✅ No hay valores nulos")

# ═══════════════════════════════════════
# PERSONAJES
# ═══════════════════════════════════════
print("\n\n👥 PERSONAJES:")
print("-" * 80)
print(f"Total registros: {len(df_characters):,}")

print(f"\n📊 Información general:")
df_characters.info()

print(f"\n❌ Valores nulos por columna:")
null_counts_chars = df_characters.isnull().sum()
null_counts_chars = null_counts_chars[null_counts_chars > 0]
if len(null_counts_chars) > 0:
    print(null_counts_chars)
else:
    print("   ✅ No hay valores nulos")

# Análisis de listas
if 'films' in df_characters.columns:
    df_characters['num_films'] = df_characters['films'].apply(
        lambda x: len(x) if isinstance(x, list) else 0
    )
    print(f"\n🌟 Top 10 personajes con más películas:")
    top_chars = df_characters.nlargest(10, 'num_films')[['name', 'num_films']]
    print(top_chars.to_string(index=False))
    
    print(f"\n📊 Estadísticas de apariciones:")
    print(f"   Promedio: {df_characters['num_films'].mean():.1f} películas")
    print(f"   Máximo: {df_characters['num_films'].max()} películas")
    print(f"   Personajes sin películas: {(df_characters['num_films'] == 0).sum()}")

🔍 EXPLORACIÓN INICIAL DE DATOS


🎬 PELÍCULAS:
--------------------------------------------------------------------------------
Total registros: 119

📊 Información general:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   film_title                          119 non-null    object 
 1   brand                               119 non-null    object 
 2   box_office_revenue                  119 non-null    object 
 3   opening_revenue                     119 non-null    object 
 4   release_date                        119 non-null    object 
 5   opening_revenue_over_total_revenue  119 non-null    int64  
 6   imdb_score                          119 non-null    float64
 7   rt_critics_score                    119 non-null    int64  
 8   rt_audience_score                   119 non-null    int64  
dtypes: f

In [12]:
# ══════════════════════════════════════════════════════════════════
# CELDA 11: GUARDAR DATOS PARA FASE 2 (PICKLE)
# ══════════════════════════════════════════════════════════════════

print("📦 GUARDANDO DATOS PARA FASE 2\n")
print("=" * 80)

# Empaquetar datos
datos_fase1 = {
    'df_movies': df_movies,
    'df_characters': df_characters,
    'metadata': {
        'movies_count': len(df_movies) if df_movies is not None else 0,
        'characters_count': len(df_characters),
        'movies_source': 'Kaggle CSV',
        'characters_source': 'Disney API',
        'api_pages_retrieved': successful_pages,
        'ingestion_date': datetime.now().isoformat(),
        'notebook': '01_ingesta_datos.ipynb',
        'status': 'SUCCESS'
    }
}

# Guardar pickle
pickle_file = 'datos_fase1.pkl'
with open(pickle_file, 'wb') as f:
    pickle.dump(datos_fase1, f)

print(f"✅ Pickle guardado: {pickle_file}")
print(f"\n📊 Contenido:")
print(f"   🎬 df_movies: {len(df_movies):,} registros")
print(f"   👥 df_characters: {len(df_characters):,} registros")
print(f"   📋 metadata: {len(datos_fase1['metadata'])} campos")

📦 GUARDANDO DATOS PARA FASE 2

✅ Pickle guardado: datos_fase1.pkl

📊 Contenido:
   🎬 df_movies: 119 registros
   👥 df_characters: 1,500 registros
   📋 metadata: 8 campos


In [13]:
# ══════════════════════════════════════════════════════════════════
# CELDA 12: VERIFICAR ARCHIVOS EN S3
# ══════════════════════════════════════════════════════════════════

print("☁️  VERIFICANDO ARCHIVOS EN S3\n")
print("=" * 80)

# Listar objetos en el bucket
try:
    print(f"📁 Contenido de s3://{S3_BUCKET}/{S3_RAW_PREFIX}/\n")
    
    response = s3_client.list_objects_v2(
        Bucket=S3_BUCKET,
        Prefix=S3_RAW_PREFIX
    )
    
    if 'Contents' in response:
        print(f"✅ {len(response['Contents'])} archivos encontrados:\n")
        
        for obj in response['Contents']:
            key = obj['Key']
            size_kb = obj['Size'] / 1024
            modified = obj['LastModified'].strftime('%Y-%m-%d %H:%M:%S')
            
            print(f"📄 {key}")
            print(f"   Tamaño: {size_kb:.1f} KB")
            print(f"   Modificado: {modified}")
            print()
    else:
        print("⚠️  No se encontraron archivos")
        
except Exception as e:
    print(f"❌ Error al listar archivos: {e}")

☁️  VERIFICANDO ARCHIVOS EN S3

📁 Contenido de s3://xideralaws-curso-fernanda/disney-project/raw/

✅ 7 archivos encontrados:

📄 disney-project/raw/api/.keep
   Tamaño: 0.0 KB
   Modificado: 2025-10-16 20:48:43

📄 disney-project/raw/api/disney_characters.csv
   Tamaño: 343.3 KB
   Modificado: 2025-10-17 16:55:24

📄 disney-project/raw/api/disney_characters.json
   Tamaño: 710.7 KB
   Modificado: 2025-10-17 16:55:24

📄 disney-project/raw/api/disney_characters_20251015_182621.json
   Tamaño: 222.6 KB
   Modificado: 2025-10-15 18:52:10

📄 disney-project/raw/kaggle/.keep
   Tamaño: 0.0 KB
   Modificado: 2025-10-16 20:48:43

📄 disney-project/raw/kaggle/disney_movies.csv
   Tamaño: 10.2 KB
   Modificado: 2025-10-17 16:54:03

📄 disney-project/raw/kaggle/disney_movies_20251015_182621.csv
   Tamaño: 10.3 KB
   Modificado: 2025-10-15 18:51:09



In [14]:
# ══════════════════════════════════════════════════════════════════
# CELDA 13: RESUMEN FINAL DEL NOTEBOOK
# ══════════════════════════════════════════════════════════════════

print("\n" + "=" * 80)
print("🎉 NOTEBOOK 01: INGESTA DE DATOS - COMPLETADO")
print("=" * 80)

print(f"\n📊 RESUMEN DE DATOS:")
print(f"   🎬 Películas: {len(df_movies):,} registros")
print(f"   👥 Personajes: {len(df_characters):,} registros")
print(f"   📡 Páginas API: {successful_pages} exitosas")

print(f"\n💾 ARCHIVOS LOCALES:")
print(f"   ✅ data/raw/kaggle/disney_movies.csv")
print(f"   ✅ data/raw/api/disney_characters.json")
print(f"   ✅ data/raw/api/disney_characters.csv")
print(f"   ✅ datos_fase1.pkl")

print(f"\n☁️  ARCHIVOS EN S3:")
print(f"   ✅ {S3_RAW_PREFIX}/kaggle/disney_movies.csv")
print(f"   ✅ {S3_RAW_PREFIX}/api/disney_characters.json")
print(f"   ✅ {S3_RAW_PREFIX}/api/disney_characters.csv")

print(f"\n🚀 SIGUIENTE PASO:")
print(f"   Ejecutar: 02_limpieza_transformacion.ipynb")

print("=" * 80)


🎉 NOTEBOOK 01: INGESTA DE DATOS - COMPLETADO

📊 RESUMEN DE DATOS:
   🎬 Películas: 119 registros
   👥 Personajes: 1,500 registros
   📡 Páginas API: 30 exitosas

💾 ARCHIVOS LOCALES:
   ✅ data/raw/kaggle/disney_movies.csv
   ✅ data/raw/api/disney_characters.json
   ✅ data/raw/api/disney_characters.csv
   ✅ datos_fase1.pkl

☁️  ARCHIVOS EN S3:
   ✅ disney-project/raw/kaggle/disney_movies.csv
   ✅ disney-project/raw/api/disney_characters.json
   ✅ disney-project/raw/api/disney_characters.csv

🚀 SIGUIENTE PASO:
   Ejecutar: 02_limpieza_transformacion.ipynb
